# SKD Pipeline Example

In [1]:
import sys, os
from datetime import datetime
# Setup
skd_python_dir = os.path.dirname(os.getcwd())
if(skd_python_dir not in sys.path):
    sys.path.append(skd_python_dir)

# Import local skd_libraries
import skd_collision_tests.collision_environment.collision_env_utils as collision_env_utils
import skd_collision_tests.collision_environment.collision_environment as collision_environment
import skd_collision_tests.collision_environment.collision_experiments_loader as collision_loader
import skd_collision_tests.collision_environment.collision_data_analyser as collision_data_analyser

# Import skd core libraries
import skd_core.skd_core_utils.skd_core_utils as skd_core_utils
import skd_core.skd_core_generators.skd_kamikaze_traj_gen as skd_kamikaze_traj_gen
import skd_core.skd_core_analysers.skd_kamikaze_data_analyser as skd_kamikaze_data_analyser

### Collision Environment Pipeline

In [2]:
# Example function to generate collision_exp config
def gen_collision_exp_config(output_dir):
    output_path = output_dir + "/collision_config.yaml"
    
    # Set optional configurations
    # Safe trajectory files to consider, if empty, then GUI will allow user to select
    safe_files = []
    # Controller ids in experiment
    multiplier_ids= [0.5, 0.625, 0.75]
    # Number of collision experiments per safe trajectory
    num_runs=10
    # Maximum number of steps simulated per experiment
    max_num_steps=25
    # Maximum number of trajectories considered per safe trajectory file
    max_trajs_per_file=-1
    # Default controller behaviour type ("basic" only for now)
    car_controller_type="basic"
    
    collision_env_utils.gen_collision_experiments_config(output_path, safe_files, num_runs, max_num_steps, max_trajs_per_file,
                                car_controller_type, multiplier_ids)
    return output_path



def run_collision_env(output_dir, config_file):
    # Create a loader for the config file
    experiments_loader = collision_loader.CollisionExperimentLoader(output_dir, config_file)

    # Run experiments from the config file
    experiments_loader.run_collision_experiments()

    # Get collision summary stats
    summary_filepath = experiments_loader.get_summary_file_path()

    return summary_filepath


def get_collision_summary_stats(output_dir, summary_stats_file):

    """ Create a collision experiment analyser """
    collision_analyser = collision_data_analyser.CollisionExperimentDataAnalyser(summary_stats_file, output_dir)

    """ Parse the data and review it on output_dir """
    collision_analyser.get_analyzer_summary_statistics()



def run_test_collision_experiments(collisions_outputdir):
    """ Excutes all three steps of the collision experiments pipeline.
    1. Gen collision experiments config
    2. Execute collision experiments
    3. Parse collision experiments results 
    """
    print("Generating collision config file")
    config_filepath = gen_collision_exp_config(collisions_outputdir)
    print("Executing collision experiments from config file")
    summary_outpath = run_collision_env(collisions_outputdir, config_filepath)
    print("Analysing collision data")
    get_collision_summary_stats(collisions_outputdir, summary_outpath)

### Kamikaze Generation Pipeline functions

In [3]:
#################################################### Pipeline for kamikaze experiments generation ####################################
# Example function to generate kamikaze traj generation experiments
def generate_kamikaze_experiments_config(kamikaze_traj_gen_outdir):
    # Set outpath for config file (Send it to ../config for now)
    # Set options for config file
    config_outpath = kamikaze_traj_gen_outdir + "/kamikaze_traj_gen.yaml"
    controller_ids = [0.5, 0.625, 0.75]
    # Safe trajectory filepaths to consider. If empty (GUI will ask user for files)
    safe_traj_files = []
    attempts_per_file = 5
    max_traj_per_file = 5
    skd_core_utils.generate_kamikaze_configs(config_outpath, controller_ids, attempts = attempts_per_file, trajs_per_file = max_traj_per_file)
    
    # Return config outpath
    return config_outpath



# Execute the kamikaze trajecoty experiments based on configuration file
def execute_kamikaze_experiments(outputdir, kamikaze_config_filepath, planner_exec_path, generic_config_path):
    # Create a generator to serve all the options in the configuration file
    kamikaze_generator = skd_kamikaze_traj_gen.KamikazeTrajGenerator(kamikaze_config_filepath, outputdir)
    kamikaze_generator.execute_kamikaze_traj_gen_configs(planner_exec_path, generic_config_path)

    return kamikaze_generator.get_experiments_summary_dir()



## Example function to analyse output from kamikaze traj experiments
def analyse_experimental_output(summary_file_path, outputdir):
    analyser = skd_kamikaze_data_analyser.SKDKamikazeDataAnalyser(summary_file_path, outputdir)
    analyser.parse_summary_data()




def run_kamikaze_traj_gen_stats(kamikaze_traj_gen_outdir, planner_path, generic_config_path):
    # Generate configuration file for experiments
    print("Generating kamikaze trajectories experiments config file")
    config_filepath = generate_kamikaze_experiments_config(kamikaze_traj_gen_outdir)

    # Execute experiments
    print("Executing kamikaze trajectories experiments")
    summary_file = execute_kamikaze_experiments(kamikaze_traj_gen_outdir, config_filepath, planner_path, generic_config_path)

    # Analyse data
    print("Processing kamikaze trajectory generation results")
    analyse_experimental_output(summary_file, kamikaze_traj_gen_outdir)


### Run pipeline

In [4]:
# Setup
# Run kamikaze trajetory generation experiments
planner_path = os.getenv("HOME") + "/SKD_DEVEL/opptv5/bin/abt"
generic_cfg_path = os.getenv("HOME") + "/SKD_DEVEL/skd_python/config/KamikazeTrajGen.cfg"
# Get timestampt
timestamp = datetime.now().strftime("%m-%d-%H-%M-%S")
module_outdir = os.getcwd() + "/testing_output/PIPE_%s" % (timestamp)


# Run collision pipeline
run_test_collision_experiments(module_outdir)


# Run kamikaze trajectory experiments
run_kamikaze_traj_gen_stats(module_outdir, planner_path, generic_cfg_path)

Generating collision config file
{'multiplier_ids': [0.5, 0.625, 0.75], 'car_controller_type': 'basic', 'num_runs': 10, 'max_num_steps': 25, 'safe_trajectory_files': ['/home/jimy/SKD_DEVEL/skd_python/skd_trajectories/skd_trajectories_db/final_experiments_trajectories.json'], 'max_trajs_per_file': -1}
Executing collision experiments from config file
{'car_controller_type': 'basic', 'max_num_steps': 25, 'max_trajs_per_file': -1, 'multiplier_ids': [0.5, 0.625, 0.75], 'num_runs': 10, 'safe_trajectory_files': ['/home/jimy/SKD_DEVEL/skd_python/skd_trajectories/skd_trajectories_db/final_experiments_trajectories.json']}
[Errno 17] File exists: '/home/jimy/SKD_DEVEL/skd_python/skd_notebooks/testing_output/PIPE_06-16-10-23-34'
Analysing collision data


/home/jimy/anaconda2/envs/skd_conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2023: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/jimy/anaconda2/envs/skd_conda/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Generating kamikaze trajectories experiments config file


TypeError: generate_kamikaze_configs() missing 1 required positional argument: 'cfg_template_path'